In [ ]:
import numpy as np
import cv2 as cv
import glob
import os
import cv2.aruco as Aruco
import json

## Set path to images/video captured of your charuco board

In [ ]:
data_path = r"example_path/to/data/videos.mp4"

In [ ]:
# Charuco board parameters
aruco_dict = Aruco.getPredefinedDictionary(Aruco.DICT_6X6_50) # Feel free to change this to other aruco markers but keep consistent throughout the project
board = Aruco.CharucoBoard((8, 5), 0.5, 0.3, aruco_dict) # Feel free to change this to other aruco markers but keep consistent throughout the project

In [ ]:
# termination criteria for refining the detected corners
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

## Steps to calibrate the camera
1. Capture images of the charuco board from different angles and distances, make sure to also cupture markers in the corners and sides of your camera - this is typically where most of the distortion occurs
2. Detect the charuco board corners and refine them
3. Store the detected corners and the corresponding object points
4. Draw the corners for visualization

In [ ]:
allCorners = []
allIds = []
decimator = 0

cap = cv.VideoCapture(data_path)
while(cap.isOpened()):

    ret, img = cap.read()
    if not ret:
        break

    # my video is 30 fps, only need 1/30 of the frames
    if decimator%30 == 0:
        # print(decimator)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        res = cv.aruco.detectMarkers(gray, aruco_dict)
        # Find the chess board corners
        if len(res[0])>0:
            
            res2 = cv.aruco.interpolateCornersCharuco(res[0],res[1], gray, board)

            if res2[1] is not None and res2[2] is not None and len(res2[1])>3:
                corners = cv.cornerSubPix(gray, res2[1], (11,11), (-1,-1), criteria)
                allCorners.append(corners)
                allIds.append(res2[2])

            cv.aruco.drawDetectedMarkers(img,res[0],res[1])
            cv.imshow('img', img)
        cv.waitKey(500)
    
    decimator+=1
cv.destroyAllWindows()

In [ ]:
# Use detected corners to compute calibration params
cal = cv.aruco.calibrateCameraCharuco(allCorners,allIds,board,gray.shape[::-1],None,None)

In [ ]:
ret, mtx, dist, rvecs, tvecs = cal # mtx is the camera matrix, dist is the distortion coefficients

In [ ]:
camera_dictionary = {'distortion': dist[0].tolist(), "intrinsic": mtx.tolist()} # convert to list so that it can be saved as json

In [ ]:
# Save everything for later
with open('camera_params.json', 'w') as outfile:
    json.dump(camera_dictionary, outfile)

In [ ]:
# test if intrinsics can be used to undistort image
img = cv.imread(r'example/path/to/image.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)